In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib
import numpy as np
import tables
from collections import Counter
from traitlets.config.loader import Config

from ctapipe.io import read_table
from ctapipe.containers import EventType

from ctapipe.instrument import CameraGeometry
from ctapipe.visualization import CameraDisplay
from ctapipe.image import (
        hillas_parameters,
        tailcuts_clean,
        apply_time_delta_cleaning
        )

from lstchain.io.config import get_cleaning_parameters
from lstchain.io.config import get_standard_config
from lstchain.io.config import read_configuration_file
from lstchain.io.io import read_camera_geometries, read_single_camera_geometry
from lstchain.io import DL1ParametersContainer, standard_config, replace_config
from lstchain.calib.camera.pixel_threshold_estimation import get_threshold_from_dl1_file
from lstchain.image.cleaning import apply_dynamic_cleaning

plt.rcParams.update({'font.size': 18})

In [ ]:
def plot_img(ax, img, geom, text_label="signal [p.e.]", text_title="Image"):
    disp = CameraDisplay(geom, ax=ax)
    disp.image = img
    disp.add_colorbar(ax=ax, label=text_label)
    disp.cmap = 'gnuplot2'
    ax.set_title(text_title, fontsize=20)
    format_axes(ax)
    
def plot_img_hillas(ax, img, geom, hillas, text_label="signal [p.e.]", text_title="Image"):
    disp = CameraDisplay(geom, ax=ax)
    disp.image = img
    disp.add_colorbar(ax=ax, label=text_label)
    disp.cmap = 'gnuplot2'
    disp.overlay_moments(hillas, color='xkcd:green', lw=3)
    ax.set_title(text_title, fontsize=20)
    format_axes(ax)
    
def format_axes(ax):
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_xlabel("")
    ax.set_ylabel("")
    

def show_images_cleaning_with_cosmic_events(dl1_path, config_file_path="", config=None, max_events=100, min_size=200):
    clean_method_name = 'tailcuts_clean_with_pedestal_threshold'
    if config_file_path:
        config = read_configuration_file(config_file_path)
    
    # read cleaning parameters 
    sigma = config[clean_method_name]['sigma']
    cleaning_params = get_cleaning_parameters(config, clean_method_name)
    pic_th, boundary_th, isolated_pixels, min_n_neighbors = cleaning_params
    
    
    pedestal_thresh = get_threshold_from_dl1_file(dl1_path, sigma)
    # Set picture threshold as array max(8, ped_mean + 2.5*ped_std) - pixel-wise
    picture_th = np.clip(pedestal_thresh, pic_th, None)
    print(f"Fraction of pixel cleaning thresholds above picture thr.:"
          f"{np.sum(pedestal_thresh>pic_th) / len(pedestal_thresh):.3f}")
    
    delta_time = config[clean_method_name]['delta_time']
    print(config[clean_method_name])

    camera_geometry = read_single_camera_geometry(dl1_path, "LSTCam")
    tablename = "/dl1/event/telescope/parameters/LST_LSTCam"
    params = read_table(dl1_path, tablename)
    
    with tables.open_file(dl1_path, mode='r') as f:
        image_table = f.root['/dl1/event/telescope/image/LST_LSTCam'][:max_events]
        ucts_trigger_type = f.root['/dl1/event/telescope/parameters/LST_LSTCam'].col('ucts_trigger_type')[:max_events]
        for ii, row in enumerate(image_table):
            image = row['image']
            peak_time = row['peak_time']
            if params['event_type'][ii] == EventType.SUBARRAY.value:
            
                signal_pixels = tailcuts_clean(camera_geometry,
                                               image,
                                               picture_thresh=picture_th,
                                               boundary_thresh=boundary_th,
                                               keep_isolated_pixels=isolated_pixels,
                                               min_number_picture_neighbors=min_n_neighbors)
            
                cleaning_img = image.copy()
                cleaning_img[~signal_pixels] = 0
                n_pixels = np.count_nonzero(signal_pixels)
            
            
                if n_pixels > 0:
                    cleaned_pixel_times = peak_time
                    cleaned_pixel_times[~signal_pixels] = np.nan
                    new_mask = apply_time_delta_cleaning(
                                                 camera_geometry,
                                                 signal_pixels,
                                                 cleaned_pixel_times,
                                                 1,
                                                 delta_time)
                    signal_pixels = new_mask
                    time_cleaning_img = cleaning_img.copy()
                    time_cleaning_img[~signal_pixels] = 0
                    cleaning_img = time_cleaning_img
        
                    n_pixels = np.count_nonzero(signal_pixels)
                    size = np.sum(cleaning_img[signal_pixels])
                
                # Show images with size > 200 p.e.
                if n_pixels > 0 and size > 200:
                    fig, ax = plt.subplots(1, 2, figsize=(14, 6))
                    plot_img(ax[0], image, camera_geometry)
                    print("i = ", ii)
                    hillas = hillas_parameters(camera_geometry[signal_pixels],
                                           cleaning_img[signal_pixels])
                    plot_img_hillas(ax[1], cleaning_img, camera_geometry, hillas, text_title="Cleaning image")
                    plt.show()

Check how pedestal tailcuts clean with delta time cleaning works with cosmic events   
* Cleaning 6-3
* Cleaning 8-4

delta time = 2 ns

# Crab

### 6-3

In [ ]:
dl1_path = "/fefs/aswg/data/real/DL1/20201120/v0.7.3/tailcut84/dl1_LST-1.Run02969.0010.h5"
config_file_63 = "/fefs/aswg/lstosa/config_files_lstchain/lstchain_standard_config.json"
show_images_cleaning_with_cosmic_events(dl1_path, config_file_path=config_file_63, max_events=20)

### 8-4

In [ ]:
dl1_path = "/fefs/aswg/data/real/DL1/20201120/v0.7.3/tailcut84/dl1_LST-1.Run02969.0010.h5"
config_file_84 = "/fefs/aswg/lstosa/config_files_lstchain/lstchain_standard_config_tailcut84.json"
show_images_cleaning_with_cosmic_events(dl1_path, config_file_path=config_file_84, max_events=20)

# Mrk 421

### 6-3

In [ ]:
dl1_path = "/fefs/aswg/data/real/DL1/20210316/v0.7.1/tailcut84/dl1_LST-1.Run04097.0005.h5"
config_file_63 = "/fefs/aswg/lstosa/config_files_lstchain/lstchain_standard_config.json"
show_images_cleaning_with_cosmic_events(dl1_path, config_file_path=config_file_63, max_events=10)

### 8-4

In [ ]:
config_84 = Config({
    "tailcuts_clean_with_pedestal_threshold": {
        'picture_thresh': 8,
        'boundary_thresh': 4, 
        'sigma': 2.5, 
        'keep_isolated_pixels': False, 
        'min_number_picture_neighbors': 2, 
        'use_only_main_island': False, 
        'delta_time': 2
    }
}) 
dl1_path = "/fefs/aswg/data/real/DL1/20210316/v0.7.1/tailcut84/dl1_LST-1.Run04097.0005.h5"
show_images_cleaning_with_cosmic_events(dl1_path, config_file_path=False, config=config_84, max_events=10)

# Dynamic cleaning

In [ ]:
config = read_configuration_file("/fefs/aswg/lstosa/config_files_lstchain/lstchain_standard_config_tailcut84_dynamic_cleaning_v075.json")

In [ ]:
config['dynamic_cleaning']

In [ ]:
dl1_path = "/fefs/aswg/data/real/DL1/20201120/v0.7.3/tailcut84/dl1_LST-1.Run02969.0010.h5"
camera_geometry = read_single_camera_geometry(dl1_path, "LSTCam")
tablename = "/dl1/event/telescope/parameters/LST_LSTCam"
params = read_table(dl1_path, tablename)


clean_method_name = 'tailcuts_clean_with_pedestal_threshold'
sigma = config[clean_method_name]['sigma']
cleaning_params = get_cleaning_parameters(config, clean_method_name)
pic_th, boundary_th, isolated_pixels, min_n_neighbors = cleaning_params
    
THRESHOLD_DYNAMIC_CLEANING = config['dynamic_cleaning']['threshold']
FRACTION_CLEANING_SIZE = config['dynamic_cleaning']['fraction_cleaning_intensity']
    
pedestal_thresh = get_threshold_from_dl1_file(dl1_path, sigma)
picture_th = np.clip(pedestal_thresh, pic_th, None)
print(f"Fraction of pixel cleaning thresholds above picture thr.:"
        f"{np.sum(pedestal_thresh>pic_th) / len(pedestal_thresh):.3f}")
    
delta_time = config[clean_method_name]['delta_time']
print(config[clean_method_name])

pedestal_thresh = get_threshold_from_dl1_file(dl1_path, sigma)
picture_th = np.clip(pedestal_thresh, pic_th, None)


with tables.open_file(dl1_path, mode='r') as f:
    image_table = f.root['/dl1/event/telescope/image/LST_LSTCam'][:10000]
    ucts_trigger_type = f.root['/dl1/event/telescope/parameters/LST_LSTCam'].col('ucts_trigger_type')[:1000]
    for ii, row in enumerate(image_table):
        image = row['image']
        peak_time = row['peak_time']
        if params['event_type'][ii] == EventType.SUBARRAY.value:
            
            signal_pixels = tailcuts_clean(camera_geometry,
                                            image,
                                            picture_thresh=picture_th,
                                            boundary_thresh=boundary_th,
                                            keep_isolated_pixels=isolated_pixels,
                                            min_number_picture_neighbors= min_n_neighbors)
            
            cleaning_img = image.copy()
            cleaning_img[~signal_pixels] = 0
            n_pixels = np.count_nonzero(signal_pixels)
            
            
            if n_pixels > 0:
                cleaned_pixel_times = peak_time
                cleaned_pixel_times[~signal_pixels] = np.nan
                new_mask = apply_time_delta_cleaning(
                                                 camera_geometry,
                                                 signal_pixels,
                                                 cleaned_pixel_times,
                                                 1,
                                                 delta_time)
                signal_pixels = new_mask
                time_cleaning_img = cleaning_img.copy()
                time_cleaning_img[~signal_pixels] = 0
                cleaning_img = time_cleaning_img
        
                n_pixels = np.count_nonzero(signal_pixels)
        
            max_3_value_index = np.argsort(image)[-3:]
            mean_3_max_signal = np.mean(image[max_3_value_index])
            if mean_3_max_signal > THRESHOLD_DYNAMIC_CLEANING:
                new_mask = apply_dynamic_cleaning(image,
                                                  signal_pixels,
                                                  THRESHOLD_DYNAMIC_CLEANING,
                                                  FRACTION_CLEANING_SIZE)
                signal_pixels = new_mask
                dynamic_cleaning_img = cleaning_img.copy()
                dynamic_cleaning_img[~signal_pixels] = 0
                cleaning_img = dynamic_cleaning_img
                
                fig, ax = plt.subplots(1, 2, figsize=(14, 6))
                plot_img(ax[0], image, camera_geometry)
                print("Size =", np.sum(cleaning_img[signal_pixels]))
                hillas = hillas_parameters(camera_geometry[signal_pixels],
                                           cleaning_img[signal_pixels])
                plot_img_hillas(ax[1], cleaning_img, camera_geometry, hillas, text_title="Cleaning image")
                plt.show()

# Exercises :
1. Show cosmic events dl1_LST-1.Run04097.0001.h5" using tailcuts clean method picture_threshold = 7, boundary_threshold=3.5 and sigma = 2.5, delta_time = 2